## Generating Compatible Distributions:

In [1]:
import numpy as np
import itertools
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

# Define the four functions

## Bin(INT), Bin(INT) -> Bin(INT)
def identity(x):
    return x

def swap(x):
    return 1-x

def to0(_):
    return 0

def to1(_):
    return 1


basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))


def generate_top(u):
    f_u, g_u = all_strategies[u]
    # print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle

    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 

    return top_level

symb = [sp.symbols(f"w{i}") for i in range(16)]
summed_strategies = sum([symb[i]*generate_top(i) for i in range(16)])
# for x in range(summed_strategies.shape[0]):
#     for y in range(summed_strategies.shape[1]):
#         sum_values = np.sum(summed_strategies[x, y])
#         summed_strategies[x, y] /= sum_values
#         summed_strategies[x, y] = np.round(summed_strategies[x, y], 3)
summed_strategies

P_AB_giv_XY = {}
for a,b,x,y in itertools.product([0,1], repeat=4):
    P_AB_giv_XY[(a,b,x,y)] = summed_strategies[x][y][a][b]
# P_AB_giv_XY

In [2]:
summed_strategies

array([[[[w0 + w10 + w2 + w8, w1 + w11 + w3 + w9],
         [w12 + w14 + w4 + w6, w13 + w15 + w5 + w7]],

        [[w10 + w4 + w6 + w8, w11 + w5 + w7 + w9],
         [w0 + w12 + w14 + w2, w1 + w13 + w15 + w3]]],


       [[[w1 + w10 + w2 + w9, w0 + w11 + w3 + w8],
         [w13 + w14 + w5 + w6, w12 + w15 + w4 + w7]],

        [[w10 + w5 + w6 + w9, w11 + w4 + w7 + w8],
         [w1 + w13 + w14 + w2, w0 + w12 + w15 + w3]]]], dtype=object)

In [3]:
def f():
    random_values = np.random.rand(16)
    num_levels = 1000
    quantized_values = np.round(random_values * (num_levels - 1)) / (num_levels - 1)
    quantized_values = quantized_values.reshape((2, 2, 2, 2))
    normalized_values = quantized_values.copy()

    for x in range(normalized_values.shape[0]):
        for y in range(normalized_values.shape[1]):
            # Calculate the sum of values for the current x, y coordinates
            sum_values = np.sum(normalized_values[x, y])
            # Normalize the values for the current x, y coordinates
            normalized_values[x, y] /= sum_values

    precision = 3  # desired precision
    normalized_values = np.round(normalized_values, precision)
    error = 4.0 - np.sum(normalized_values)
    normalized_values[0, 0, 0, 0] += error
    normalized_values[0, 0, 0, 0] += 4 - np.sum(normalized_values)
    normalized_values = np.round(normalized_values, precision)
    normalized_values

    P_AB_giv_XY = {}
    for a,b,x,y in itertools.product([0,1], repeat=4):
        P_AB_giv_XY[(a,b,x,y)] = normalized_values[x][y][a][b]
    # sum(P_AB_giv_XY.values())

    # print sum of all values at different a,b when x,y is fixed from P_AB_giv_XY
    for x,y in itertools.product([0,1], repeat=2):
        sum = 0
        for a,b in itertools.product([0,1], repeat=2):
            # print(f"P({a},{b}|{x},{y}) = {P_AB_giv_XY[(a,b,x,y)]}")
            sum += P_AB_giv_XY[(a,b,x,y)]
        print(f"x={x}, y={y}", f"sum = {sum}")
        

In [4]:
def g():
    random_values = np.random.rand(16)
    num_levels = 1000
    quantized_values = np.round(random_values * (num_levels - 1)) / (num_levels - 1)
    quantized_values = quantized_values.reshape((2, 2, 2, 2))
    normalized_values = quantized_values.copy()

    for x in range(normalized_values.shape[0]):
        for y in range(normalized_values.shape[1]):
            sum_values = np.sum(normalized_values[x, y])
            normalized_values[x, y] /= sum_values
            normalized_values[x, y] = np.round(normalized_values[x, y], 3)

    for x in range(normalized_values.shape[0]):
        for y in range(normalized_values.shape[1]):
            sum_values = np.sum(normalized_values[x, y])
            if sum_values != 1.0:
                idx = np.where(normalized_values[x, y] != 0)[0][0]
                normalized_values[x, y, idx] += 1.0 - sum_values

    # in dict form
    P_AB_giv_XY = {}
    for a,b,x,y in itertools.product([0,1], repeat=4):
        P_AB_giv_XY[(a,b,x,y)] = normalized_values[x][y][a][b]
    print("\n", P_AB_giv_XY)

    # sum all values in P_AB_giv_XY
    print("\n", sum(P_AB_giv_XY.values()))

    # prob dist
    print("\n", normalized_values)

In [5]:
def generate_dist():
    f()
    g()

In [6]:
generate_dist()

x=0, y=0 sum = 0.9990000000000001
x=0, y=1 sum = 1.0
x=1, y=0 sum = 0.9999999999999999
x=1, y=1 sum = 1.001

 {(0, 0, 0, 0): 0.324, (0, 0, 0, 1): 0.266, (0, 0, 1, 0): 0.1329999999999999, (0, 0, 1, 1): 0.247, (0, 1, 0, 0): 0.155, (0, 1, 0, 1): 0.318, (0, 1, 1, 0): 0.1959999999999999, (0, 1, 1, 1): 0.325, (1, 0, 0, 0): 0.398, (1, 0, 0, 1): 0.273, (1, 0, 1, 0): 0.263, (1, 0, 1, 1): 0.207, (1, 1, 0, 0): 0.123, (1, 1, 0, 1): 0.143, (1, 1, 1, 0): 0.409, (1, 1, 1, 1): 0.221}

 4.0009999999999994

 [[[[0.324 0.155]
   [0.398 0.123]]

  [[0.266 0.318]
   [0.273 0.143]]]


 [[[0.133 0.196]
   [0.263 0.409]]

  [[0.247 0.325]
   [0.207 0.221]]]]
